In [7]:
######### 始める前に設定必要の変数：###########

#### 1. BT : Academic申請後に生成される Bearer Token(BT)
#### 2. TWEET_LIMIT_you_set : 一回のクエリーの上限
#### 3. queryword_path : 辞書のエクセルファイル
#### 4. savedir_you_set : csvの保存フォルダー

BT = '##########'

TWEET_LIMIT_you_set = 100000                   ####  テストするときには1000とか設定した方がいいです
                                            #### 毎月の上限MAXは 10,000,000 なので　9,900,000にすることがおすすめです
savedir_you_set = './'

In [11]:
import requests
import time
import pandas as pd


import datetime
from pytz import timezone



search_url = "https://api.twitter.com/2/tweets/counts/all"




# Optional params: start_time,end_time,since_id,until_id,next_token,granularity
def make_parm(query, start_time, end_time):

    query_params = {'query': query ,
                    'granularity': 'day',
                    'start_time': start_time,
                    'end_time': end_time,
                    #'max_results': 500,  #一回のqueryは５００で上限らしい
                    'next_token' : {} #次のページにいくためのparam
                   }
    return query_params



def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FullArchiveTweetCountsPython"
    return r


def create_headers(BT):
    headers = {"Authorization": "Bearer {}".format(BT)}
    return headers


def connect_to_endpoint(url, headers,params):
    response = requests.request("GET", search_url, params=params,headers=headers)
    #print(response.status_code)
    #print(response.json())
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response


def store_checkpoint(normalized_data, query):
    
    # store collected data to csv in case of error occurrence
    
    savedir = savedir_you_set
    
    kwindex = query.split()[0] ## name the file by the first keyword
    
    sorted_data = normalized_data.sort_values(by = 'start').reset_index(drop = True)
        
    filename = kwindex + '-' + sorted_data.iloc[0]['start'][:10] + '-' + \
               sorted_data.iloc[-1]['start'][:10] + \
                '.csv'
        
    savepath = savedir + filename    
    
    sorted_data.to_csv(path_or_buf = savepath)
    print('Successfully saved at ' + savepath + ' at ' + \
              datetime.datetime.today().strftime("%Y/%m/%d-%H:%M"))
    

    
def json_to_df(json_response):
    
    normalized_data = pd.json_normalize(json_response['data'])
    
    return normalized_data
    


#実際にツイート取得をする関数(上の5つの関数を呼び出しながら)
def main(query, start_time, end_time):

    count = 0
    flag = True
    TWEET_LIMIT = TWEET_LIMIT_you_set
    
    
    normalized_data_old = pd.DataFrame(columns=['end',
                                                                                        'start', 
                                                                                        'tweet_count'])
    
    #cols = ['created_at', 'id', ' text']
    #normalized_data_old = pd.DataFrame(index=[], columns=cols)#create checkpoint dataframe
    
    query_params = make_parm(query, start_time, end_time)#ここで取得条件が返される
    
    while flag:
        if count >= TWEET_LIMIT:
            break
    
        headers = create_headers(BT)
        time.sleep(1)
        
        response = connect_to_endpoint(search_url, headers, query_params)
        
        if response.status_code != 200:
            print(str(response.status_code) + 'Error occurred. Now trying to save csv')
            normalized_data_last = convert_timezone(normalized_data_old)
            store_checkpoint(normalized_data_last, query)
        
        json_response = response.json()  
        
        normalized_data = json_to_df(json_response)
        
        normalized_data_new = pd.concat([normalized_data_old, normalized_data], ignore_index=True)
         
        normalized_data_old = normalized_data_new
        
        print("total:" + str(len(normalized_data_old)) + "tweets")
        if count % 5000 <= 1000:
            print('Finished collecting tweets at ' + str(normalized_data_old.iloc[-1]['start']))  
    

        result_count = json_response['meta']['total_tweet_count']
        
        if 'next_token' in json_response['meta']:
            next_token = json_response['meta']['next_token']
            query_params['next_token'] = next_token
            count += result_count
            
            time.sleep(3)
            
            response = connect_to_endpoint(search_url, headers, query_params)
            time.sleep(3)
            
            
            if response.status_code == 200:
                
                time.sleep(1)
                
                continue
            
            elif response.status_code == 429:
                
                # store checkpoints when error occurs
                
                normalized_data_last = convert_timezone(normalized_data_old)
                store_checkpoint(normalized_data_last, keyword)
                
                # from https://github.com/mammalofski/Twitter-Scraper/blob/main/Scraper.py
                
                print('too many requests ... ')
                print('the header is ', response.headers)
                throttle_end_timestamp = int(response.headers.get('x-rate-limit-reset'))
                throttle_end_time = datetime.strftime(datetime.fromtimestamp(throttle_end_timestamp), "%H:%M:%S")
                time_to_wait = int(throttle_end_timestamp - datetime.now().timestamp()) + 10
                print('lets rest for', time_to_wait, 'seconds and wake up at', throttle_end_time)
                print('sleeping ...')
                time.sleep(time_to_wait)
                response = connect_to_endpoint(search_url, headers, query_params)
                
            else:
                normalized_data_last = convert_timezone(normalized_data_old)
                store_checkpoint(normalized_data_last, keyword)
                
                print('un expected error : ')
                print(response.status_code, response.text)
                print('sleep for 15 min')
                time.sleep( 15 * 60 + 10)  # rest 15min 
                print('restart at ' + datetime.strftime(datetime.now(), "%Y-%m-%d %H:%M:%S"))
                
                response = connect_to_endpoint(search_url, headers, query_params)
             
            time.sleep(3)  # rate limit = 1 request/1 sec
            
            json_response = response.json()
            
            
            #append the current page of results to lists
            
            
            normalized_data = json_to_df(json_response)
        
            normalized_data_new = pd.concat([normalized_data_old, normalized_data], ignore_index=True)
            
            normalized_data_old = normalized_data_new
            
            print("total:" + str(len(normalized_data_old)) + "tweets")
            if count % 5000 <= 1000:
                print('Finished collecting tweets at ' + str(normalized_data_old.iloc[-1]['start']))  
    

        else:
            flag = False
            print('Last page of query results')
            
    
    normalized_data_last = normalized_data_old

    # store checkpoint to csv
    
    store_checkpoint(normalized_data_last, query)
    
    #normalized_data_last.to_csv('../FullArchiveData_2021Apr/T20200331-0501.csv')#csvファイルに保存

In [9]:
query = '研究楽しい'

In [12]:
if __name__ == "__main__":
        main(query, '2022-01-01T00:00:00Z','2022-01-10T00:00:00Z')

total:9tweets
Finished collecting tweets at 2022-01-09T00:00:00.000Z
Last page of query results
Successfully saved at ./研究楽しい-2022-01-01-2022-01-09.csv at 2022/03/24-11:50
